In [1]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from datetime import datetime 
import math
import time


In [2]:
pd.options.display.max_rows = 10000


In [10]:

#  parameters
# %env PROVIDER_API_KEY=YOUR_API_KEY_HERE
circle="zone"
month="aug2023"
month_doc=8
month_unpaid=10

date='DEC_2023'
ftpHost = 'ftp.uppclonline.com'
ftpUname = 'mvftpreport'
ftpPass = 'Mvftp@321'
localFolderPath = f".\{month}\{circle}\master"
remoteFolder = "MASTER_DATA/DEC_2023"

In [11]:
# import ftplib
# import os
# import gzip
# import shutil
# import time



# def downloadFilesFromFtp(localfolderPath, targetFilenames, ftpHost, ftpUname, ftpPass, remoteWorkingDirectory):
#     # initialize the flag that specifies if download is success
#     isDownloadSuccess: bool = False

#     # create an FTP client instance, use the timeout parameter for slow connections only
#     ftp = ftplib.FTP(timeout=30)
#     ftp.encoding = "utf-8"

#     # connect to the FTP server
#     ftp.connect(ftpHost)

#     # login to the FTP server
#     ftp.login(ftpUname, ftpPass)

#     # change current working directory if specified
#     if not (remoteWorkingDirectory == None or remoteWorkingDirectory.strip() == ""):
#         _ = ftp.cwd(remoteWorkingDirectory)

#     # iterate through each remote filename and download
#     for fItr in range(len(targetFilenames)):
#         targetFilename = targetFilenames[fItr]
#         # derive the local file path by appending the local folder path with remote filename
#         localFilePath = os.path.join(localfolderPath, targetFilename)
     
#         print("downloading file {0}".format(targetFilename))
#         # download FTP file using retrbinary function
#         with open(localFilePath, "wb") as file:
#             retCode = ftp.retrbinary("RETR " + targetFilename, file.write)
         
#         with gzip.open(localFilePath, 'rb') as f_in:
#             with open(localFilePath[:-3], 'wb') as f_out:
#                 shutil.copyfileobj(f_in, f_out)
#         os.remove(localFilePath)
#     # read file
            
#     # send QUIT command to the FTP server and close the connection
#     ftp.quit()

#     # check if download is success using the return code (retCode)
#     if retCode.startswith('226'):
#         isDownloadSuccess = True
#     return isDownloadSuccess



# remoteFilenames = [ f'MASTER_MVVNL_DIV354921_{date}.csv.gz', f'MASTER_MVVNL_DIV354922_{date}.csv.gz', f'MASTER_MVVNL_DIV354923_{date}.csv.gz', f'MASTER_MVVNL_DIV355131_{date}.csv.gz', f'MASTER_MVVNL_DIV355132_{date}.csv.gz', f'MASTER_MVVNL_DIV355133_{date}.csv.gz', f'MASTER_MVVNL_DIV355211_{date}.csv.gz', f'MASTER_MVVNL_DIV355212_{date}.csv.gz', f'MASTER_MVVNL_DIV355213_{date}.csv.gz', f'MASTER_MVVNL_DIV355214_{date}.csv.gz']

# # run the function to download the files from FTP server
# isDownloadSuccess = downloadFilesFromFtp(
#     localFolderPath,remoteFilenames, ftpHost,ftpUname, ftpPass, remoteFolder)

# print("download status = {0}".format(isDownloadSuccess))

In [12]:
def loop(file_name):
    return pd.read_csv(f"{month}/{circle}/master/{file_name}",on_bad_lines='skip')

In [13]:
import os

mylist = os.listdir(f"{month}/{circle}/master")

In [14]:
def bifurcation(column_name,area):
    #area = [Circle,Division]
# column_name = 'Country'
# replace_symbols = ['>', '<', ':', '"', '/', '\\\\', '\|', '\?', '\*']
# df[column_name] = (
#     df[column_name].replace(replace_symbols, '', regex=True).str.strip().str.title()
# )

    os.mkdir(f'{month}/{circle}/master_{area}') 
    unique_values = df[column_name].unique()
    print(unique_values)
    for unique_value in unique_values:
        start = time.time()
        print(unique_value)
        df_output = df[df[column_name].str.contains(unique_value)]
        output_path = os.path.join(f'{month}/{circle}/master_{area}', str(unique_value) + '.csv')
        df_output.to_csv(output_path, index=False)
        end = time.time()
        print("CSV:", end - start)

In [15]:
mylist

['MASTER_MVVNL_DIV354921_AUG_2023.csv',
 'MASTER_MVVNL_DIV354922_AUG_2023.csv',
 'MASTER_MVVNL_DIV354923_AUG_2023.csv',
 'MASTER_MVVNL_DIV355131_AUG_2023.csv',
 'MASTER_MVVNL_DIV355132_AUG_2023.csv',
 'MASTER_MVVNL_DIV355133_AUG_2023.csv',
 'MASTER_MVVNL_DIV355211_AUG_2023.csv',
 'MASTER_MVVNL_DIV355212_AUG_2023.csv',
 'MASTER_MVVNL_DIV355213_AUG_2023.csv',
 'MASTER_MVVNL_DIV355214_AUG_2023.csv']

In [16]:
def NonGOVT(df):
    convert_dict = {'SUPPLY_TYPE': str,
                
                }
 
    df = df.astype(convert_dict)
    df['SUPPLY_TYPE'] = df['SUPPLY_TYPE'].astype(str).str.replace('\\.0+$', '')
    govt = ['30','31','32','33','34','35','36','37','38','39','40','40A', '41', '41A', '42', '42A', '43', '44', '45','48','72','73','74','75','80','81','82','83','100','101','102','103','104','105','106','107','80B','42A','41A','40A','H31'
           'H32','H33','H41','H42','H43','HV41','H13','H14']
    govt = [str(x) for x in govt]
    df['gov_flag'] = df['SUPPLY_TYPE'].isin(govt)
    return df

In [17]:
def relGov(df):
#    ((df['gov_flag'] == False) & ( (pd.isnull(df['GOVT_CODE'] )))) 
    df['relGov'] = 'Goverment'
    df.loc[(pd.isnull(df['GOVT_CODE'] ))  ,"relGov"] = 'Non-Goverment'
    return df

In [18]:
def arrear(df):
    
#     df = df[df['TOTAL_OUTSTANDING'] > 0]
    df['ARREAR_FILTER'] = None
    # df.loc[(df['TOTAL_OUTSTANDING'] < 5000) ,"ARREAR_FILTER"] = "<5k"
    # df.loc[((df['TOTAL_OUTSTANDING'] >= 5000) & (df['TOTAL_OUTSTANDING'] < 100000)),"ARREAR_FILTER"] = "Between 5K-25K	"
    # df.loc[((df['TOTAL_OUTSTANDING'] >= 25000) & (df['TOTAL_OUTSTANDING'] < 50000)),"ARREAR_FILTER"] = "Between 25K-50K"
    # df.loc[((df['TOTAL_OUTSTANDING'] >= 50000) & (df['TOTAL_OUTSTANDING'] < 100000)),"ARREAR_FILTER"] = "Between 50K-1Lac"
    # df.loc[df['TOTAL_OUTSTANDING'] >= 100000,"ARREAR_FILTER"] = "Above 1Lac"
    df.loc[(df['arrear_withot_Lpsc'] < 5000) ,"ARREAR_FILTER"] = "<5k"
    df.loc[((df['arrear_withot_Lpsc'] >= 5000) & (df['arrear_withot_Lpsc'] < 100000)),"ARREAR_FILTER"] = "Between 5K-25K"
    df.loc[((df['arrear_withot_Lpsc'] >= 25000) & (df['arrear_withot_Lpsc'] < 50000)),"ARREAR_FILTER"] = "Between 25K-50K"
    df.loc[((df['arrear_withot_Lpsc'] >= 50000) & (df['arrear_withot_Lpsc'] < 100000)),"ARREAR_FILTER"] = "Between 50K-1Lac"
    df.loc[df['arrear_withot_Lpsc'] >= 100000,"ARREAR_FILTER"] = "Above 1Lac"
    
    
    return df

In [19]:

def low_consumtion_filter(df):
    
#     df = df[df['TOTAL_OUTSTANDING'] > 0]
    df['low_consumtion_filter'] = None
    df.loc[df['CONSUMPTION_CURR_MNTH'] <= 10,"low_consumtion_filter"] = "0-10"
    df.loc[((df['CONSUMPTION_CURR_MNTH'] > 10 ) & (df['CONSUMPTION_CURR_MNTH'] <= 30)),"low_consumtion_filter"] = "10-30"
    df.loc[((df['CONSUMPTION_CURR_MNTH'] > 30) & (df['CONSUMPTION_CURR_MNTH'] <= 50)),"low_consumtion_filter"] = "30-50"
    df.loc[((df['CONSUMPTION_CURR_MNTH'] > 50) & (df['CONSUMPTION_CURR_MNTH'] <=100)),"low_consumtion_filter"] = "50-100"
    df.loc[df['CONSUMPTION_CURR_MNTH'] > 100,"low_consumtion_filter"] = "> 100"
    
    return df

In [46]:
def life_line(df):
    
#     df = df[df['TOTAL_OUTSTANDING'] > 0]
    df['life_line'] = False
#     df.loc[df['CONSUMPTION_CURR_MNTH'] <= 10,"low_consumtion_filter"] = "0-10"
#     df.loc[((df['CONSUMPTION_CURR_MNTH'] > 10 ) & (df['CONSUMPTION_CURR_MNTH'] <= 30)),"low_consumtion_filter"] = "10-30"
#     df.loc[((df['CONSUMPTION_CURR_MNTH'] > 30) & (df['CONSUMPTION_CURR_MNTH'] <= 50)),"low_consumtion_filter"] = "30-50"
    df.loc[((df['Cat'] == 'LMV1') & (df['CONSUMPTION_CURR_MNTH'] <=100) & (df['CON_STATUS'] == 'In Service') & (df['LOAD'] <= 1)),"life_line"] = True
#     df.loc[df['CONSUMPTION_CURR_MNTH'] > 100,"low_consumtion_filter"] = "> 100"
    
    return df

In [21]:
def mdi_cat(mdi,load):
    result=""
    
    if (math.isnan(mdi) or math.isnan(load) ):
        return None
    
    elif mdi <= 0.25*load:
        result = "less than 25% of load"
        return(result)
    elif mdi <= .5*load:
        result = "less than 50% of load and grtr than 25%"
        return(result)
    elif mdi <= load:
        result = "less than 100% of load and grtr than 50%"
        return(result)
    elif mdi<=1.50*load:
        result = "greter than 100% of load and less than 150%"
        return(result)
    elif (mdi<=2*load):
        result = "greter than 150% of load and less than 200%"
        return(result)
    return("greter than 200%")
    
        
    
        
    



def MDI_filter(df):
    
#     df = df[df['TOTAL_OUTSTANDING'] > 0]
    df['MDI_filter'] = df.apply(lambda row : mdi_cat(row['MDI'],row['LOAD']), axis = 1)
    
    
    
    return df

In [22]:
def billable(df):
    bil = ['In Service','TD','TD >6 MONTHS']
    df['bilable_flag'] = df['CON_STATUS'].isin(bil)
    return df
  

In [23]:
def CatLoad(df):
    df['LOAD'] = pd.to_numeric(df['LOAD'])   #,errors="coerce"
    df['LOAD_FILTER'] = None
    df.loc[df['LOAD'] < 5,"LOAD_FILTER"] = "1-5kw"
    df.loc[((df['LOAD'] >= 5) & (df['LOAD'] < 10)),"LOAD_FILTER"] = "5-9kw"
    df.loc[df['LOAD'] >= 10,"LOAD_FILTER"] = "great than 10kw"
    return df


In [24]:
def rdf_cat(df):
    cat246=['LMV2','LMV4B','LMV6']
    # df.Cat.isin(cat)
    df['LOAD'] = pd.to_numeric(df['LOAD'])   #,errors="coerce"
    df['rdf_cat'] = None
    df.loc[(df.Cat.isin(cat246)),"rdf_cat"] = "LMV2,LMV4B,LMV6"
    df.loc[((df['LOAD'] >= 3) & (df['Cat']=='LMV1')),"rdf_cat"] = "LMV1(3-9kw)"
    df.loc[((df['Cat']=='LMV1') & (df['LOAD'] <3)),"rdf_cat"] = "LMV1(less than 3kw)"
    # df.loc[((df['SUPPLY_TYPE'] == '17') & (df['LOAD'] <3)),"rdf_cat"] = "LMV1(rural and < 3kw)"
    # df.loc[((df['LOAD'] >= 5) & (df['LOAD'] < 10)),"rdf_cat"] = "5-9kw"
    df.loc[df['LOAD'] >= 10,"rdf_cat"] = "great than 10kw"
    df.loc[~df['rdf_cat'].notna(),"rdf_cat"] = "other(LMV3,LMV4A,LMV7)"
    
    
    return df


In [25]:
def Cat_VC(df):
    df['LOAD'] = pd.to_numeric(df['LOAD'])   #,errors="coerce"
    df['Cat_vc'] = None
    df.loc[((df['LOAD'] >= 1) & (df['Cat']=='LMV2')),"Cat_vc"] = "LMV2(1-5kw)"
    df.loc[((df['LOAD'] >= 3) & (df['Cat']=='LMV1')),"Cat_vc"] = "LMV1(3-5kw)"
    df.loc[((df['Cat']=='LMV1') & (df['LOAD'] <3)),"Cat_vc"] = "LMV1(urban and < 3kw)"
    df.loc[((df['SUPPLY_TYPE'] == '17') & (df['LOAD'] <3)),"Cat_vc"] = "LMV1(rural and < 3kw)"
    
    df.loc[((df['LOAD'] >= 5) & (df['LOAD'] < 10)),"Cat_vc"] = "5-9kw"
    df.loc[df['LOAD'] >= 10,"Cat_vc"] = "great than 10kw"
    return df


In [26]:
def new_cat(df):
    df['LOAD'] = pd.to_numeric(df['LOAD'])   #,errors="coerce"
    df['new_cat'] = None
    df.loc[((df['LOAD'] >= 1) & (df['Cat']=='LMV1')),"new_cat"] = "LMV1(2-5kw)"
    df.loc[((df['LOAD'] >= 5) & (df['Cat']=='LMV1')),"new_cat"] = "LMV1(5-9kw)"
    df.loc[((df['LOAD'] >= 1) & (df['Cat']=='LMV2')),"new_cat"] = "LMV2(2-5kw)"
    df.loc[((df['LOAD'] >= 5) & (df['Cat']=='LMV2')),"new_cat"] = "LMV2(5-9kw)"
    df.loc[((df['LOAD'] >= 1) & (df['Cat']=='LMV4A')),"new_cat"] = "LMV4(2-5kw)"
    df.loc[((df['LOAD'] >= 5) & (df['Cat']=='LMV4A')),"new_cat"] = "LMV4(5-9kw)"
    df.loc[((df['LOAD'] >= 1) & (df['Cat']=='LMV4B')),"new_cat"] = "LMV4(2-5kw)"
    df.loc[((df['LOAD'] >= 5) & (df['Cat']=='LMV4B')),"new_cat"] = "LMV4(5-9kw)"
    df.loc[df['LOAD'] >= 10,"new_cat"] = "great than 10kw"
    return df


In [27]:
def graph_VC(df):
    # df['LOAD'] = pd.to_numeric(df['LOAD'])   #,errors="coerce"
    df['graph_vc'] = None
    df.loc[(df['Cat']=='LMV2'),"graph_vc"] = "LMV2(urban)"
    df.loc[(df['Cat']=='LMV1'),"graph_vc"] = "LMV1(urban)"
    df.loc[(df['SUPPLY_TYPE'] == '17'),"graph_vc"] = "LMV1(rural)"
    df.loc[(df['SUPPLY_TYPE'] == '27'),"graph_vc"] = "LMV2(rural)"
    df.loc[(((df['Cat']=='HV1') | (df['Cat']=='HV2') | (df['Cat']=='LMV6')) & (df.gov_flag == False)) ,"graph_vc"] = "HV1,HV2 and LMV6"
    df.loc[(((df['Cat']=='LMV4B') | (df['Cat']=='LMV5') | (df['Cat']=='LMV9'))) ,"graph_vc"] = "Other(4b,5,9)"
    df.loc[( df.gov_flag == True) ,"graph_vc"] = "Government"
    
    # df.loc[((df['LOAD'] >= 5) & (df['LOAD'] < 10)),"Cat_vc"] = "5-9kw"
    # df.loc[df['LOAD'] >= 10,"Cat_vc"] = "great than 10kw"
    return df


In [28]:
def ots(df):
    # df['LOAD'] = pd.to_numeric(df['LOAD'])   #,errors="coerce"
    df['ots_vc'] = None
    df['discount_Nov'] = None
    df['discount_Dec(1-15)'] = None
    df['Registration_amount'] = None
    df['Registration_amount'] = 0.3*df['TOTAL_OUTSTANDING']
    df['Amount_to_be_paid_after_disc_Nov'] = None
    df['Amount_to_be_paid_after_disc_dec(1-15)'] = None
    
    df.loc[((df['Cat']=='LMV1') & (df['LOAD'] <= 1)),"ots_vc"] = "LMV1 And <=1kwh"
    df.loc[((df['Cat']=='LMV1')& (df['LOAD'] > 1)),"ots_vc"] = "LMV1 And >1kwh"
    df.loc[((df['Cat']=='LMV2') & (df['LOAD'] <= 3)),"ots_vc"] = "LMV2 And <=3kwh"
    df.loc[((df['Cat']=='LMV2')& (df['LOAD'] > 3)),"ots_vc"] = "LMV2 And >3kwh"
    df.loc[(df['Cat']=='LMV4B'),"ots_vc"] = "LMV4B"
    df.loc[(df['Cat']=='LMV5'),"ots_vc"] = "LMV5"
    df.loc[(df['Cat']=='LMV6'),"ots_vc"] = "LMV6"
    df.loc[(df['ots_vc']=='LMV1 And <=1kwh'),"discount_Nov"] = df[df['ots_vc']=='LMV1 And <=1kwh']['LPSC']*1
    df.loc[(df['ots_vc']=='LMV1 And <=1kwh'),"discount_Dec(1-15)"] = df[df['ots_vc']=='LMV1 And <=1kwh']['LPSC']*1
    df.loc[(df['ots_vc']=='LMV1 And >1kwh'),"discount_Nov"] = df[df['ots_vc']=='LMV1 And >1kwh']['LPSC']*0.9
    df.loc[(df['ots_vc']=='LMV1 And >1kwh'),"discount_Dec(1-15)"] = df[df['ots_vc']=='LMV1 And >1kwh']['LPSC']*0.8
    df.loc[(df['ots_vc']=='LMV2 And <=3kwh'),"discount_Nov"] = df[df['ots_vc']=='LMV2 And <=3kwh']['LPSC']*0.8
    df.loc[(df['ots_vc']=='LMV2 And <=3kwh'),"discount_Dec(1-15)"] = df[df['ots_vc']=='LMV2 And <=3kwh']['LPSC']*0.7
    df.loc[(df['ots_vc']=='LMV2 And >3kwh'),"discount_Nov"] = df[df['ots_vc']=='LMV2 And >3kwh']['LPSC']*0.6
    df.loc[(df['ots_vc']=='LMV2 And >3kwh'),"discount_Dec(1-15)"] = df[df['ots_vc']=='LMV2 And >3kwh']['LPSC']*0.5
    df.loc[(df['ots_vc']=='LMV4B'),"discount_Dec(1-15)"] = df[df['ots_vc']=='LMV4B']['LPSC']*0.5
    df.loc[(df['ots_vc']=='LMV4B'),"discount_Nov"] = df[df['ots_vc']=='LMV4B']['LPSC']*0.4
    df.loc[(df['ots_vc']=='LMV5'),"discount_Dec(1-15)"] = df[df['ots_vc']=='LMV5']['LPSC']*1
    df.loc[(df['ots_vc']=='LMV5'),"discount_Nov"] = df[df['ots_vc']=='LMV5']['LPSC']*1
    df.loc[(df['ots_vc']=='LMV6'),"discount_Dec(1-15)"] = df[df['ots_vc']=='LMV6']['LPSC']*0.5
    df.loc[(df['ots_vc']=='LMV6'),"discount_Nov"] = df[df['ots_vc']=='LMV6']['LPSC']*0.4
    
    df['Amount_to_be_paid_after_disc_Nov'] = df['TOTAL_OUTSTANDING']-df['discount_Nov']
    df['Amount_to_be_paid_after_disc_dec(1-15)'] = df['TOTAL_OUTSTANDING']-df['discount_Dec(1-15)']
    df['arrear_withot_Lpsc'] = df['TOTAL_OUTSTANDING']-df['LPSC']
    

    
    
    # df.loc[((df['LOAD'] >= 5) & (df['LOAD'] < 10)),"Cat_vc"] = "5-9kw"
    # df.loc[df['LOAD'] >= 10,"Cat_vc"] = "great than 10kw"
    return df


In [29]:
def paid_arrear(df):
    df["paid_arrear"] = 0
    df["temp"] = 0
    df.loc[(df['CURRENT_ASSESSMENT']>=0),"temp"] = df[df['CURRENT_ASSESSMENT']>=0]['CURRENT_ASSESSMENT'] - df[df['CURRENT_ASSESSMENT']>=0]['TOTAL_PAY_AMT']
    df.loc[(df['temp']<0),"paid_arrear"] = -1 * (df[df['temp']<0]["temp"])
    df.drop('temp', inplace=True, axis=1)
    return df

In [30]:
# def div(df):
#        #,errors="coerce"
#     df['Division'] = None
#     df.loc[((df['LOAD'] >= 1) & (df['Cat']=='LMV2')),"Division"] = "LMV2(1-5kw)"
#     df.loc[((df['LOAD'] >= 3) & (df['Cat']=='LMV1')),"Cat_vc"] = "LMV1(3-5kw)"
    
#     df.loc[((df['LOAD'] >= 5) & (df['LOAD'] < 10)),"Cat_vc"] = "5-9kw"
#     df.loc[df['LOAD'] >= 10,"Cat_vc"] = "great than 10kw"
#     return df


In [31]:
def Division_Name(df):
    # df['LOAD'] = pd.to_numeric(df['LOAD'])   #,errors="coerce"
    
    # df['Circle_Name'] = None
    df.insert(loc=0, column="Circle_Name", value=None)
    # df['Division_Name'] = None
    df.insert(loc=1, column="Division_Name", value=None)
    
    
    
    df.loc[df['DIV_CODE']== 'DIV354921',"Division_Name"] = "EDD I Bahraich"
    df.loc[df['DIV_CODE']== 'DIV354921',"Circle_Name"] = "EDC Bahraich"
    df.loc[df['DIV_CODE']== 'DIV354922',"Division_Name"] = "EDD II Nanpara"
    df.loc[df['DIV_CODE']== 'DIV354922',"Circle_Name"] = "EDC Bahraich"
    df.loc[df['DIV_CODE']== 'DIV354923',"Division_Name"] = "EDD III Kaisarganj"
    df.loc[df['DIV_CODE']== 'DIV354923',"Circle_Name"] = "EDC Bahraich"
    df.loc[df['DIV_CODE']== 'DIV355211',"Division_Name"] = "EDD I Gonda"
    df.loc[df['DIV_CODE']== 'DIV355211',"Circle_Name"] = "EDC Gonda"
    df.loc[df['DIV_CODE']== 'DIV355212',"Division_Name"] = "EDD II Gonda"
    df.loc[df['DIV_CODE']== 'DIV355212',"Circle_Name"] = "EDC Gonda"
    df.loc[df['DIV_CODE']== 'DIV355213',"Division_Name"] = "EDD III COLONELGANJ"
    df.loc[df['DIV_CODE']== 'DIV355213',"Circle_Name"] = "EDC Gonda"
    df.loc[df['DIV_CODE']== 'DIV355214',"Division_Name"] = "EDD IV MANKAPUR"
    df.loc[df['DIV_CODE']== 'DIV355214',"Circle_Name"] = "EDC Gonda"
    df.loc[df['DIV_CODE']== 'DIV355131',"Division_Name"] = "EDD Balrampur"
    df.loc[df['DIV_CODE']== 'DIV355131',"Circle_Name"] = "EDC Balrampur"
    df.loc[df['DIV_CODE']== 'DIV355132',"Division_Name"] = "EDD SHRAWASTI"
    df.loc[df['DIV_CODE']== 'DIV355132',"Circle_Name"] = "EDC Balrampur"
    df.loc[df['DIV_CODE']== 'DIV355133',"Division_Name"] = "EDD Tulsipur"
    df.loc[df['DIV_CODE']== 'DIV355133',"Circle_Name"] = "EDC Balrampur"

    return df
    

In [32]:
def faulty_Ageing(df):
    df['METER_READ_FLTY_CNT'] = pd.to_numeric(df['METER_READ_FLTY_CNT'])       #,errors="coerce"
    df['faulty_Ageing'] = None
    df.loc[df['METER_READ_FLTY_CNT'] < 2,"faulty_Ageing"] = "faulty<2"
    df.loc[((df['METER_READ_FLTY_CNT'] >= 2) & (df['METER_READ_FLTY_CNT'] < 3)),"faulty_Ageing"] = "faulty>=2 TO <3"
    df.loc[df['METER_READ_FLTY_CNT'] >= 3,"faulty_Ageing"] = "faulty >=3"
    return df


In [33]:
def categories(df):
    #Supplytype.srs.startswith('1') for lmv1
    convert_dict = {'SUPPLY_TYPE': str,
                
                }
 
    df = df.astype(convert_dict)
    df['SUPPLY_TYPE'] = df['SUPPLY_TYPE'].astype(str).str.replace('\\.0+$', '')
    df['Cat'] = None
#     df[df['SUPPLY_TYPE'].str.startswith('1')] = "LMV1"
    df.loc[df['SUPPLY_TYPE'].str.startswith('1', na=False),"Cat"] = "LMV1"
    df.loc[df['SUPPLY_TYPE'].str.startswith('2',na=False),"Cat"] = "LMV2"
    df.loc[df['SUPPLY_TYPE'].str.startswith('3', na=False),"Cat"] = "LMV3"
    # df.loc[df['SUPPLY_TYPE'].str.startswith('4', na=False),"Cat"] = "LMV4"
    df.loc[df['SUPPLY_TYPE'].str.startswith('5', na=False),"Cat"] = "LMV5"
    df.loc[df['SUPPLY_TYPE'].str.startswith('6', na=False),"Cat"] = "LMV6"
    df.loc[df['SUPPLY_TYPE'].str.startswith('7', na=False),"Cat"] = "LMV7"
    df.loc[df['SUPPLY_TYPE'].str.startswith('8', na=False),"Cat"] = "LMV8"
    df.loc[df['SUPPLY_TYPE'].str.startswith('9', na=False),"Cat"] = "LMV9"
#     df.loc[df['SUPPLY_TYPE'].str.startswith('10'),"Cat"] = "LMV10"
    df.loc[df['SUPPLY_TYPE'].str.startswith('H1', na=False),"Cat"] = "HV1"
    df.loc[df['SUPPLY_TYPE'].str.startswith('H2', na=False),"Cat"] = "HV2"
    df.loc[df['SUPPLY_TYPE'].str.startswith('H3', na=False),"Cat"] = "HV3"
    df.loc[df['SUPPLY_TYPE'].str.startswith('H4', na=False),"Cat"] = "HV4"
    lmv10 = ['100', '101', '103', '104', '105']
    # df['gov_flag'] = df['SUPPLY_TYPE'].isin(lmv10)
    df.loc[df['SUPPLY_TYPE'].isin(lmv10),"Cat"] = "LMV10"
    lmv4a = ['40','40A', '41', '41A', '42', '42A', '43', '44', '45','48']
    lmv4b = ['46', '47']
    df.loc[df['SUPPLY_TYPE'].isin(lmv4a),"Cat"] = "LMV4A"
    df.loc[df['SUPPLY_TYPE'].isin(lmv4b),"Cat"] = "LMV4B"
    return df

In [34]:
def type(df):
    df['LAST_PAY_DATE'] = pd.to_datetime(df['LAST_PAY_DATE'],dayfirst=True,errors ='coerce')
    df['DOC'] = pd.to_datetime(df['DOC'],dayfirst=True,errors ='coerce')
    df['LAST_BILL_DATE'] = pd.to_datetime(df['LAST_BILL_DATE'],dayfirst=True,errors ='coerce')
    df['LAST_OK_READ_STATUS'] = pd.to_datetime(df['LAST_OK_READ_STATUS'],dayfirst=True,errors ='coerce')
    df['TOTAL_OUTSTANDING'] = pd.to_numeric(df['TOTAL_OUTSTANDING'],errors="coerce")
    df['LOAD'] = pd.to_numeric(df['LOAD'],errors="coerce")
    df['CONSUMPTION_CURR_MNTH'] = pd.to_numeric(df['CONSUMPTION_CURR_MNTH'],errors="coerce")
    df['MDI'] = pd.to_numeric(df['MDI'],errors="coerce")
    df['CURRENT_ASSESSMENT'] = pd.to_numeric(df['CURRENT_ASSESSMENT'],errors="coerce")
    df['TOTAL_PAY_AMT'] = pd.to_numeric(df['TOTAL_PAY_AMT'],errors="coerce")
    df['LAST_PAY_AMT'] = pd.to_numeric(df['LAST_PAY_AMT'],errors="coerce")
    
    return df

In [35]:
def unpaid(df):
    df['paid_status'] = 0
    # df.loc[df['LAST_PAY_DATE'] < pd.Timestamp(2023, 1, month_unpaid),"unpaid_status"] = 0
    # df.loc[(pd.isnull(df['LAST_PAY_DATE'] )),"unpaid_status"] = 0     
    df.loc[df['LAST_PAY_DATE'] >= pd.Timestamp(2023, month_unpaid,1),"paid_status"] = 1
    
    return df

In [36]:
def newconection(df):
    df['newconection'] = 0
    # df.loc[df['LAST_PAY_DATE'] < pd.Timestamp(2023, 1, month_unpaid),"unpaid_status"] = 0
    # df.loc[(pd.isnull(df['LAST_PAY_DATE'] )),"unpaid_status"] = 0     
    df.loc[df['DOC'] >= pd.Timestamp(2022, 1,1),"newconection"] = 1
    
    return df

In [37]:
def conection_month(df):
    df['conection_month'] = "till_Aug"
    # df.loc[df['LAST_PAY_DATE'] < pd.Timestamp(2023, 1, month_unpaid),"unpaid_status"] = 0
    # df.loc[(pd.isnull(df['LAST_PAY_DATE'] )),"unpaid_status"] = 0     
    df.loc[df['DOC'] >= pd.Timestamp(2023, 9,1),"conection_month"] = "New Connection"
    # df.loc[((df['DOC'] >= pd.Timestamp(2023, 9,1)) & (df['DOC']< pd.Timestamp(2023, 10,1))),"conection_month"] = "Sep"
    # df.loc[((df['DOC'] >= pd.Timestamp(2023, 10,1)) & (df['DOC']< pd.Timestamp(2023, 11,1))),"conection_month"] = "Oct"
    # df.loc[((df['DOC'] >= pd.Timestamp(2023, 11,1)) & (df['DOC']< pd.Timestamp(2023, 12,1))),"conection_month"] = "Nov"
    
    return df

In [38]:
def Neverpaid(df):
    df['Neverpaid_count'] = 0
    df.loc[((pd.isnull(df['LAST_PAY_DATE'] )) & (df['DOC'] < pd.Timestamp(2023,month_doc, 1))),"Neverpaid_status"] = 1
    # df.loc[pd.isnull(df['Neverpaid_status']),"Neverpaid_status"] = 0
    
    return df

In [39]:
start = time.time()
df = []
df = Parallel(n_jobs=-1, verbose=10,prefer="threads")(delayed(loop)(file_name) for file_name in mylist)
df = pd.concat(df, ignore_index=True)


end = time.time()
print("CSV:", end - start)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
C:\Users\HP\AppData\Local\Temp\ipykernel_21180\2667646776.py:2: DtypeWarning: Columns (3,11,14,17,20,51,52,82) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f"{month}/{circle}/master/{file_name}",on_bad_lines='skip')
C:\Users\HP\AppData\Local\Temp\ipykernel_21180\2667646776.py:2: DtypeWarning: Columns (3,82) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f"{month}/{circle}/master/{file_name}",on_bad_lines='skip')
C:\Users\HP\AppData\Local\Temp\ipykernel_21180\2667646776.py:2: DtypeWarning: Columns (3,82) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f"{month}/{circle}/master/{file_name}",on_bad_lines='skip')
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:   14.7s remaining:   34.3s
C:\Users\HP\AppData\Local\Temp\ipykernel_21180\2667646776.py:2: DtypeWarnin

CSV: 28.97943902015686


In [91]:
# for x in mylist:
#     df = loop(x)
#     div = ['DIV354921', 'DIV354922','DIV354923', 'DIV355131',
#        'DIV355132', 'DIV355133', 'DIV355211', 'DIV355212', 'DIV355213', 'DIV355214']
#     df.drop(df.index[~df["DIV_CODE"].isin(div)],axis=0,inplace=True)
    

In [92]:
types=df.dtypes

In [93]:
types

DIV_CODE                          object
SDO_CODE                          object
ACCT_ID                          float64
KNO                               object
MOBILE_NO                        float64
LANDLINE_NO                      float64
BOOK_NO                           object
SCNO                              object
NAME                              object
ADDRESS                           object
SUPPLY_TYPE                       object
LOAD                             float64
LOAD_UNIT                         object
DOC                               object
SECURITY_AMT                      object
CON_STATUS                        object
SERIAL_NBR                        object
MULTIPLY_FACTOR                  float64
METER_STATUS                      object
LAST_BILL_DATE                    object
CLOSE_READING                     object
MDI                               object
BILL_BASIS                        object
BILL_TYP                          object
CONSUMPTION_CURR

In [94]:
df.CONSUMER_BASE.unique()

array(['URBAN', 'RURAL', nan], dtype=object)

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1521793 entries, 0 to 1521792
Columns: 122 entries, DIV_CODE to INOP_AMT
dtypes: float64(47), object(75)
memory usage: 1.4+ GB


In [40]:
df = df[df['CONSUMER_BASE'].notna()]

In [97]:
df.columns

Index(['DIV_CODE', 'SDO_CODE', 'ACCT_ID', 'KNO', 'MOBILE_NO', 'LANDLINE_NO',
       'BOOK_NO', 'SCNO', 'NAME', 'ADDRESS',
       ...
       'VILLAGE_NAME', 'HABITAT_CODE', 'HABITAT_NAME', 'DISTRICT',
       'TEMP_DISCON_DATE', 'CROSS_SUBSIDY', 'PTW_SUBSIDY', 'TARIFF_SUBSIDY',
       'POWER_LOOM_SUBSIDY', 'INOP_AMT'],
      dtype='object', length=122)

In [98]:
df.DIV_CODE.unique()

array(['DIV354921', 'DIV354922', 'DIV354923', 'DIV355131', 'DIV355132',
       'DIV355133', 'DIV355211', 'DIV355212', 'DIV355213', 'DIV355214'],
      dtype=object)

In [41]:
div = ['DIV354921', 'DIV354922','DIV354923', 'DIV355131',
       'DIV355132', 'DIV355133', 'DIV355211', 'DIV355212', 'DIV355213', 'DIV355214']
df.drop(df.index[~df["DIV_CODE"].isin(div)],axis=0,inplace=True)


In [100]:
df.DIV_CODE.unique()

array(['DIV354921', 'DIV354922', 'DIV354923', 'DIV355131', 'DIV355132',
       'DIV355133', 'DIV355211', 'DIV355212', 'DIV355213', 'DIV355214'],
      dtype=object)

In [101]:
df.SDO_CODE.unique()

array(['SDO3549212', 'SDO3549211', 'SDO3249329', 'SDO3549222',
       'SDO3549223', 'SDO3549221', 'SDO3549224', 'SDO3549231',
       'SDO3549234', 'SDO3549237', 'SDO3549232', 'SDO3549236',
       'SDO3551311', 'SDO3551312', 'SDO3551321', 'SDO3551323',
       'SDO3551324', 'SDO3551322', 'SDO3551331', 'SDO3551336',
       'SDO3551334', 'SDO3551335', 'SDO3551332', 'SDO3552111',
       'SDO3552113', 'SDO3552112', 'SDO3552124', 'SDO3552122',
       'SDO3552125', 'SDO3552121', 'SDO3552123', 'SDO3552131',
       'SDO3552132', 'SDO3552142', 'SDO3552143', 'SDO3552141'],
      dtype=object)

In [102]:
df[df["SDO_CODE"] == 'SDO3249329' ]

,DIV_CODE,SDO_CODE,ACCT_ID,KNO,MOBILE_NO,LANDLINE_NO,BOOK_NO,SCNO,NAME,ADDRESS,...,VILLAGE_NAME,HABITAT_CODE,HABITAT_NAME,DISTRICT,TEMP_DISCON_DATE,CROSS_SUBSIDY,PTW_SUBSIDY,TARIFF_SUBSIDY,POWER_LOOM_SUBSIDY,INOP_AMT
2284,DIV354921,SDO3249329,6.566506e+09,NaN,8.052109e+09,NaN,' 324932902501,MV_0120249,RAJESH KUMAR GUPTA S/O RAM LAKHAN GUPTA,STEELGANJ TALAB BAHRAICH Bahraich UP-271801 IND,...,NaN,NaN,NaN,BAHRAICH,NaN,0.0,0.0,0.0,0.0,NaN
3365,DIV354921,SDO3249329,7.615839e+09,NaN,8.090600e+09,NaN,' 324932902501,NaN,SRI PRAKASH SONI S/O SRI UMASHANKAR SONI,QAZIPURA NEAR PACHKAUDI HOTEL BAHRAICH Bahraic...,...,NaN,NaN,NaN,BAHRAICH,NaN,0.0,0.0,0.0,0.0,NaN
5582,DIV354921,SDO3249329,9.065903e+09,NaN,8.874604e+09,NaN,' 324932912721,NaN,SMT RINKU DEVI W/O MANOJ KUMAR,HAMJAPURA NEAR HANUMAN PRASAD YADAV BAHRAICH...,...,NaN,NaN,NaN,BAHRAICH,NaN,0.0,0.0,0.0,0.0,NaN
5606,DIV354921,SDO3249329,3.408189e+09,NaN,9.450426e+09,NaN,' 324932912711,NaN,IRFAN AHMAD S/O KURBAN AHMAD,NOORUDDINCHAK DARGAH SHARIF BAHRAICH Bahraich ...,...,NaN,NaN,NaN,BAHRAICH,NaN,0.0,0.0,0.0,0.0,NaN
5648,DIV354921,SDO3249329,8.798896e+09,NaN,NaN,NaN,' 324932912618,MV_064554,MOHD GUFRAN,KAZIPURA ' BAHRAICH Bahraich UP-271801 IND,...,NaN,NaN,NaN,BAHRAICH,NaN,0.0,0.0,0.0,0.0,NaN
5714,DIV354921,SDO3249329,7.239130e+09,NaN,6.390595e+09,NaN,' 324932912613,NaN,ARMAAN ALI S/O JALEEL AHMAD,SALARGANJ NEAR GHASIYARI MASJID BAHRAICH Bah...,...,NaN,NaN,NaN,BAHRAICH,NaN,0.0,0.0,0.0,0.0,NaN
8385,DIV354921,SDO3249329,3.834376e+09,NaN,NaN,NaN,' 324932902501,MV_0120346,MITHLESH KUMARI PANDEY W/O RAM KUMAR PANEY,JHURIYA SEKHDAHIR BAHRAICH Bahraich UP-271801 IND,...,NaN,NaN,NaN,BAHRAICH,NaN,0.0,0.0,0.0,0.0,NaN
9043,DIV354921,SDO3249329,8.220708e+09,NaN,1.000000e+10,NaN,' 324932902501,MV_0109141,KARINA W/O MOHD SHARIF,MAHRAAJ GAON BAXIPURA KALPIPARA BAHRAICH Bahr...,...,NaN,NaN,NaN,BAHRAICH,NaN,0.0,0.0,0.0,0.0,NaN
11549,DIV354921,SDO3249329,5.077762e+09,NaN,9.451406e+09,NaN,' 324932912735,NaN,RICHA SHUKLA,000 SINGHAPARASI PAHADA FAKKAD BAHRAICH BAHRAI...,...,NaN,NaN,NaN,BAHRAICH,NaN,-17.0,0.0,0.0,0.0,NaN
14620,DIV354921,SDO3249329,1.840163e+09,NaN,9.450425e+09,NaN,' 324932902501,MV_0120394,HABEEB AHMAd S/O HAZI DARGAHI,SUFIPURA BAHRAICH Bahraich UP-271801 IND,...,NaN,NaN,NaN,BAHRAICH,NaN,0.0,0.0,0.0,0.0,NaN


In [103]:
df.info(verbose = True,null_counts = True)


C:\Users\HP\AppData\Local\Temp\ipykernel_24392\3229557052.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  df.info(verbose = True,null_counts = True)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1521791 entries, 0 to 1521792
Data columns (total 122 columns):
 #    Column                         Non-Null Count    Dtype  
---   ------                         --------------    -----  
 0    DIV_CODE                       1521791 non-null  object 
 1    SDO_CODE                       1521791 non-null  object 
 2    ACCT_ID                        1521791 non-null  float64
 3    KNO                            1416958 non-null  object 
 4    MOBILE_NO                      1406586 non-null  float64
 5    LANDLINE_NO                    21 non-null       float64
 6    BOOK_NO                        1521791 non-null  object 
 7    SCNO                           1455677 non-null  object 
 8    NAME                           1521263 non-null  object 
 9    ADDRESS                        1521791 non-null  object 
 10   SUPPLY_TYPE                    1517280 non-null  object 
 11   LOAD                           1502258 non-null  float64
 12 

In [104]:
# df =Division_Name(df)  

In [105]:
# # df["Division_Name"].unique()
# df = df[(df.bilable_flag == True) & (df.LPSC > 1) & (df.ots_vc.isin(['LMV2 And >3kwh', 'LMV4B', 'LMV5', 'LMV6']))]

In [106]:
# bifurcation("Division_Name",'Division')   
# "Division_Name"
# "Circle_Name"


In [107]:
# df = type(df)
# df = categories(df)
# df = new_cat(df)

In [108]:
# df =Division_Name(df)

In [45]:
df['SUBSTATION'].value_counts()

COLONELGANJ RURAL_355213                     54302
FAKHARPUR_354923                             41559
MOTIPUR_354922                               35772
KAISERGANJ_354923                            33046
TULSIPUR DEHAT_355133                        31054
UTRAULA DEHAT_355133                         31020
MANKAPUR_355214                              30768
NAWABGANJ_355212                             29338
TARABGANJ_355212                             28913
URLAHAWA_355132                              28546
MASKANWA_355214                              28050
PARASPUR_355212                              26846
ARYA NAGAR_355213                            26169
MAHSI_354922                                 25711
BELSAR_355212                                25403
PAYAGPUR_354923                              25055
BHINGA_355132                                24814
JAIL ROAD GONDA_30170                        24205
ITIATHOK_355214                              24040
GHARIGHAT_355214               

In [110]:
df.shape

(1521791, 122)

In [111]:
column =list(df.columns)

In [112]:
short_col = ['ACCT_ID','KNO','DIV_CODE','Division_Name','Cat','CONSUMPTION_CURR_MNTH','LOAD','MDI','CON_STATUS']

In [42]:
im_col = ['DIV_CODE',
 'SDO_CODE',
          'SUBSTATION',
          'FEEDER',
 'ACCT_ID',
 'KNO',
          
    
          'NAME',
          'ADDRESS',
          'MOBILE_NO',
          'SUPPLY_TYPE',
          'CONSUMPTION_CURR_MNTH',
          'CONSUMPTION_PREV_MNTH',
          'DOC',
          'BILL_BASIS',
          'METER_STATUS',
          'METER_READ_REMARK',
          'BILL_TYP',
          'SBM_MACHINE_ID',
          'CON_STATUS',
          'LOAD',
           'MDI',

          'ARREAR',
          'CURRENT_ASSESSMENT',
          'TOTAL_OUTSTANDING',
          'LPSC',
          'LAST_OK_READING',
          'LAST_OK_READ_STATUS',
          'METER_READ_FLTY_CNT',
          'LAST_PAY_AMT',
          'LAST_PAY_DATE',
          'INDUSTRIAL_FLAG',
          'INF_BILL',
          'ACCT_INFLALTED_FLAG',
          'GOVT_CODE',
          'LAST_BILL_DATE',
          'PAYMENT_SRCE',
          'TOTAL_PAY_AMT',
         'CONSUMER_BASE']

In [43]:
df = df[im_col]

In [44]:
df.dropna(subset = ['ACCT_ID'], inplace=True)

In [45]:
df = type(df)
df = unpaid(df)
df=Neverpaid(df)
df = NonGOVT(df)
# df = relGov(df)
# df = faulty_Ageing(df)
df = billable(df)
# graph_VC(df)
# df = MDI_filter(df)

# df = low_consumtion_filter(df)
# df = arrear(df)
df = categories(df)
# df = ots(df)
# df = arrear(df)
df=CatLoad(df)
df=Cat_VC(df)
df=graph_VC(df)
df =Division_Name(df)
df=paid_arrear(df)
df = conection_month(df)
df = life_line(df)
df = newconection(df)

C:\Users\HP\AppData\Local\Temp\ipykernel_21180\2198998957.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['SUPPLY_TYPE'] = df['SUPPLY_TYPE'].astype(str).str.replace('\\.0+$', '')
C:\Users\HP\AppData\Local\Temp\ipykernel_21180\1802877464.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['SUPPLY_TYPE'] = df['SUPPLY_TYPE'].astype(str).str.replace('\\.0+$', '')


In [ ]:
# file_jul = "./load/urban/merged_data_urban.xlsx"
# df = newconection(df)

In [72]:
# df = rdf_cat(df)
# df_load = pd.read_excel(file_jul)

In [80]:
# df1 = df[df['MDI'] > df['LOAD']]

In [81]:
# df1 = df1[short_col]

In [82]:
# df1['month'] = month

In [83]:
# # Performing a left merge based on the 'key' column
# merged_df = pd.merge(df_load, df1, on='ACCT_ID', how='outer',suffixes=('', '_aug'))

# print("Left Merged DataFrame:")
# # print(merged_df_left)

Left Merged DataFrame:


C:\Users\HP\AppData\Local\Temp\ipykernel_19460\3903627788.py:2: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  merged_df = pd.merge(df_load, df1, on='ACCT_ID', how='outer',suffixes=('', '_aug'))


In [84]:
# # df=CatLoad(df)
# # Save the merged dataframe to a new CSV file
# merged_file = "./load/urban/merged_data_tillAug.xlsx"
# merged_df.to_excel(merged_file, index=False)

In [39]:
df['PAYMENT_SRCE']=df['PAYMENT_SRCE'].str.strip()


In [ ]:
# df=Cat_VC(df)

df.PAYMENT_SRCE.value_counts()

In [ ]:
df.columns

In [ ]:
# df = df[(df.bilable_flag==True) & (df.ots_vc.notna())]

In [43]:
# df = df[df['TOTAL_OUTSTANDING'] >= 50000]


In [47]:
div_connection=df[(df.bilable_flag == True)].pivot_table(index=['Circle_Name','Division_Name'], columns=['conection_month'],values=['ACCT_ID','LOAD'],
                    aggfunc={'ACCT_ID':'count','LOAD':np.sum},fill_value=0 )


In [47]:
div_life_line=df[(df.life_line == True)].pivot_table(index=['Circle_Name','Division_Name'], columns=['CONSUMER_BASE'],values=['ACCT_ID','LOAD'],
                    aggfunc={'ACCT_ID':'count','LOAD':np.sum},fill_value=0 )



In [50]:
div_life_line1=df[(df.Cat == 'LMV1') & (df.LOAD <= 1)].pivot_table(index=['Circle_Name','Division_Name'], columns=['CONSUMER_BASE'],values=['ACCT_ID','LOAD'],
                    aggfunc={'ACCT_ID':'count','LOAD':np.sum},fill_value=0 )


In [51]:
div_life_line1.to_excel(f"dec2023/{circle}/report/{circle}_{month}_life_line1.xlsx")


In [139]:
# df1 = df[(df.life_line == True) & ((df.CONSUMER_BASE == 'URBAN'))]

In [140]:
# df1.to_excel(f"dec2023/{circle}/report/{circle}_{month}_life_line_list.xlsx")


In [91]:
# div_connection.to_excel(f"nov2023/{circle}/report/{circle}_{month}_connection_aug.xlsx")

In [74]:
cat = ["LMV2","LMV4A","LMV4B","LMV6"]
div_bill246=df[(df.Cat.isin(cat)) & (df.bilable_flag==True)].pivot_table(index=['Circle_Name','Division_Name'],columns=["Cat"],values=['ACCT_ID'],
                    aggfunc={'ACCT_ID':'count'},fill_value=0 )
div_bill246.to_excel(f"{month}/{circle}/report/{circle}_{month}_246.xlsx")

In [48]:
cat = ["LMV9"]
div_bill9=df[(df.Cat.isin(cat)) & (df.bilable_flag==True)].pivot_table(index=['Circle_Name','Division_Name','CON_STATUS'],columns=["Cat"],values=['ACCT_ID','CURRENT_ASSESSMENT','TOTAL_OUTSTANDING','TOTAL_PAY_AMT','paid_status'],
                    aggfunc={'ACCT_ID':'count','CURRENT_ASSESSMENT':np.sum,'TOTAL_OUTSTANDING':np.sum,'TOTAL_PAY_AMT':np.sum,'paid_status':np.sum},fill_value=0 )
div_bill9.to_excel(f"{month}/{circle}/report/{circle}_{month}_9_2.xlsx")

In [42]:
# cat = ['LMV1', 'LMV2', 'LMV6', 'LMV8', 'LMV10', 'LMV9',
#        'LMV5', 'HV2', 'HV1', 'LMV4A', 'HV4']
# # cat=['LMV4A']
cat = ["LMV2,LMV4B,LMV6","LMV1(3-9kw)","great than 10kw"]

In [49]:
# div_payment_4a_nov = df[((df.paid_status == 1)&(df.Cat.isin(cat)))].pivot_table(
#     index=['LAST_PAY_DATE'], columns=['Circle_Name', 'Division_Name'], values=['TOTAL_PAY_AMT'],
#     aggfunc={'TOTAL_PAY_AMT': np.sum}, fill_value=0
# )

In [ ]:
div_payment_4a_nov = df[((df.paid_status == 1)&(df.Cat.isin(cat)))].pivot_table(
    index=['LAST_PAY_DATE'], columns=['Circle_Name', 'Division_Name'], values=['TOTAL_PAY_AMT'],
    aggfunc={'TOTAL_PAY_AMT': np.sum}, fill_value=0
)

In [44]:
div_sub = df[df.bilable_flag == True].pivot_table(
    index=['Circle_Name', 'Division_Name','SDO_CODE','SUBSTATION'], values=['ACCT_ID'],
    aggfunc={'ACCT_ID':'count'}, fill_value=0
)

In [49]:
rdf_div=df[(df.bilable_flag == True) &(df.METER_READ_REMARK== 'RDF')].pivot_table(index=['Circle_Name','Division_Name'],columns=['rdf_cat','CON_STATUS'],values=['ACCT_ID'],
                    aggfunc={'ACCT_ID':'count'},fill_value=0,margins=True,margins_name='Total' )

In [50]:
idf_div=df[(df.bilable_flag == True) &(df.METER_READ_REMARK== 'IDF')].pivot_table(index=['Circle_Name','Division_Name'],columns=['rdf_cat','CON_STATUS'],values=['ACCT_ID'],
                    aggfunc={'ACCT_ID':'count'},fill_value=0,margins=True,margins_name='Total' )

In [48]:
rdf_list = df[(df.bilable_flag == True) &(df.METER_READ_REMARK== 'RDF')&(df.rdf_cat.isin(cat))]

In [49]:
idf_list = df[(df.bilable_flag == True) &(df.METER_READ_REMARK== 'IDF')&(df.rdf_cat.isin(cat))]

In [50]:
div_payment_4a_nov.to_excel(f"{month}/{circle}/report/{circle}_{month}_4a_payment.xlsx")

In [45]:
rdf_div.to_excel(f"{month}/{circle}/report/{circle}_{month}_rdf.xlsx")

NameError: name 'rdf_div' is not defined

In [45]:
div_sub.to_excel(f"{month}/{circle}/report/{circle}_{month}_sub.xlsx")

In [50]:
rdf_list.to_excel(f"{month}/{circle}/report/{circle}_{month}_rdf_list.xlsx")

In [ ]:
idf_div.to_excel(f"{month}/{circle}/report/{circle}_{month}_idf.xlsx")

In [51]:
idf_list.to_excel(f"{month}/{circle}/report/{circle}_{month}_idf_list.xlsx")

In [132]:
# # df =Division_Name(df)
#     df['LOAD'] = pd.to_numeric(df['LOAD'])   #,errors="coerce"
#     df['LOAD_FILTER'] = None
#     df.loc[df['LOAD'] < 5,"LOAD_FILTER"] = "1-5kw"
#     df.loc[((df['LOAD'] >= 5) & (df['LOAD'] < 10)),"LOAD_FILTER"] = "5-9kw"
#     df.loc[df['LOAD'] >= 10,"LOAD_FILTER"] = "great than 10kw"

ValueError: cannot insert Circle_Name, already exists

In [42]:
# df1=df[df['LOAD_FILTER']=='great than 10kw']

In [51]:
df.dtypes

Circle_Name                      object
Division_Name                    object
DIV_CODE                         object
SDO_CODE                         object
SUBSTATION                       object
FEEDER                           object
ACCT_ID                         float64
KNO                              object
NAME                             object
ADDRESS                          object
MOBILE_NO                       float64
SUPPLY_TYPE                      object
CONSUMPTION_CURR_MNTH           float64
CONSUMPTION_PREV_MNTH            object
DOC                      datetime64[ns]
BILL_BASIS                       object
METER_STATUS                     object
METER_READ_REMARK                object
BILL_TYP                         object
SBM_MACHINE_ID                   object
CON_STATUS                       object
LOAD                            float64
MDI                             float64
ARREAR                          float64
CURRENT_ASSESSMENT              float64


In [81]:
start = time.time()
# df1.to_excel(f"{month}/top10-td-rapdrp.xlsx")
df.to_csv(f"{month}/{circle}/{circle}_oct23_ots.csv") 

end = time.time()
print("CSV:", end - start)

CSV: 76.53425645828247


In [39]:
# start = time.time()
# # df1.to_excel(f"{month}/top10-td-rapdrp.xlsx")
# df.to_csv(f"{month}/{circle}/{circle}sep23_8oct.csv") 

# end = time.time()
# print("CSV:", end - start)

CSV: 53.3649525642395


In [42]:
df.new_cat.value_counts()

LMV1(2-5kw)        1341922
LMV2(2-5kw)          56055
LMV4(2-5kw)          19702
great than 10kw       9912
LMV2(5-9kw)           2662
LMV1(5-9kw)           1661
LMV4(5-9kw)            786
Name: new_cat, dtype: int64

In [88]:
df = df[df.new_cat.notna()]

In [104]:

bifurcation("Division_Name",'Division_raw')  
# "Division_Name"
# "Circle_Name"


['EDD I Bahraich' 'EDD II Nanpara' 'EDD III Kaisarganj' 'EDD Balrampur'
 'EDD SHRAWASTI' 'EDD Tulsipur' 'EDD I Gonda' 'EDD II Gonda'
 'EDD III COLONELGANJ' 'EDD IV MANKAPUR']
EDD I Bahraich
CSV: 12.913344621658325
EDD II Nanpara
CSV: 22.742069244384766
EDD III Kaisarganj
CSV: 21.657413721084595
EDD Balrampur
CSV: 9.60334825515747
EDD SHRAWASTI
CSV: 15.763208627700806
EDD Tulsipur
CSV: 22.403606414794922
EDD I Gonda
CSV: 9.498378038406372
EDD II Gonda
CSV: 16.380317449569702
EDD III COLONELGANJ
CSV: 13.315404415130615
EDD IV MANKAPUR
CSV: 16.226884603500366


In [ ]:
df.columns

In [46]:
rdf_sdo=df[(df.bilable_flag == True) & (df.gov_flag == False) &(df.METER_READ_REMARK== 'RDF')].pivot_table(index=['Circle_Name','Division_Name','SDO_CODE','SUBSTATION'],columns=['CON_STATUS'],values=['ACCT_ID'],
                    aggfunc={'ACCT_ID':'count'},fill_value=0,margins=True,margins_name='Total' )


In [112]:
meter_read_remark =df.pivot_table(index=['Circle_Name','Division_Name',],columns=['METER_READ_REMARK','CON_STATUS'],values=['ACCT_ID'],
                    aggfunc={'ACCT_ID':'count'},fill_value=0,margins=True,margins_name='Total' )

In [113]:
BILL_BASIS =df.pivot_table(index=['Circle_Name','Division_Name',],columns=['BILL_BASIS','CON_STATUS'],values=['ACCT_ID'],
                    aggfunc={'ACCT_ID':'count'},fill_value=0,margins=True,margins_name='Total' )

In [132]:
Sbm =df[df.BILL_TYP=='SBM'].pivot_table(index=['Circle_Name','Division_Name','SBM_MACHINE_ID'],values=['ACCT_ID'],
                    aggfunc={'ACCT_ID':'count'},fill_value=0,margins=True,margins_name='Total' )

In [114]:
# pivot_table.plot(kind='pie',
#             subplots=True,
#               title='Population (1955-2020)',autopct='%1.0f%%')

Catvc=df[(df.bilable_flag == True) & (df.gov_flag == False)].pivot_table(index=['Circle_Name','Division_Name',], columns=['Cat_vc'],values=['ACCT_ID','CURRENT_ASSESSMENT','TOTAL_OUTSTANDING','TOTAL_PAY_AMT','paid_status'],
                    aggfunc={'ACCT_ID':'count','CURRENT_ASSESSMENT':np.sum,'TOTAL_OUTSTANDING':np.sum,'TOTAL_PAY_AMT':np.sum,'paid_status':np.sum},fill_value=0,margins=True,margins_name='Total' )


In [115]:
# pivot_table.plot(kind='pie',
#             subplots=True,
#               title='Population (1955-2020)',autopct='%1.0f%%')

graphvc=df[(df.bilable_flag == True)].pivot_table(index=['Circle_Name','Division_Name',], columns=['graph_vc'],values=['ACCT_ID','CURRENT_ASSESSMENT','TOTAL_OUTSTANDING','CONSUMPTION_CURR_MNTH','TOTAL_PAY_AMT','paid_status','LOAD'],
                    aggfunc={'ACCT_ID':'count','CURRENT_ASSESSMENT':np.sum,'TOTAL_OUTSTANDING':np.sum,'CONSUMPTION_CURR_MNTH':np.sum,'TOTAL_PAY_AMT':np.sum,'LOAD':np.sum,'paid_status':np.sum},fill_value=0,margins=True,margins_name='Total' )


In [50]:
df.columns

Index(['Circle_Name', 'Division_Name', 'DIV_CODE', 'SDO_CODE', 'SUBSTATION',
       'FEEDER', 'ACCT_ID', 'KNO', 'NAME', 'ADDRESS', 'MOBILE_NO',
       'SUPPLY_TYPE', 'CONSUMPTION_CURR_MNTH', 'CONSUMPTION_PREV_MNTH', 'DOC',
       'BILL_BASIS', 'METER_STATUS', 'METER_READ_REMARK', 'BILL_TYP',
       'SBM_MACHINE_ID', 'CON_STATUS', 'LOAD', 'MDI', 'ARREAR',
       'CURRENT_ASSESSMENT', 'TOTAL_OUTSTANDING', 'LPSC', 'LAST_OK_READING',
       'LAST_OK_READ_STATUS', 'METER_READ_FLTY_CNT', 'LAST_PAY_AMT',
       'LAST_PAY_DATE', 'INDUSTRIAL_FLAG', 'ACCT_INFLALTED_FLAG', 'GOVT_CODE',
       'LAST_BILL_DATE', 'PAYMENT_SRCE', 'TOTAL_PAY_AMT', 'CONSUMER_BASE',
       'paid_status', 'Neverpaid_count', 'Neverpaid_status', 'gov_flag',
       'bilable_flag', 'Cat', 'LOAD_FILTER', 'Cat_vc', 'graph_vc',
       'paid_arrear', 'conection_month', 'life_line', 'newconection'],
      dtype='object')

In [42]:
# pivot_table.plot(kind='pie',
#             subplots=True,
#               title='Population (1955-2020)',autopct='%1.0f%%')

unpaid_new=df[(df.bilable_flag == True) & (df.gov_flag == False) & (df.newconection == 1) & (df.paid_status == 0)].pivot_table(index=['Circle_Name','Division_Name',], columns=['INF_BILL','BILL_BASIS'],values=['ACCT_ID','CURRENT_ASSESSMENT','TOTAL_OUTSTANDING','CONSUMPTION_CURR_MNTH'],
                    aggfunc={'ACCT_ID':'count','CURRENT_ASSESSMENT':np.sum,'TOTAL_OUTSTANDING':np.sum,'CONSUMPTION_CURR_MNTH':np.sum},fill_value=0)


In [44]:
unpaid_new.to_excel(f"{month}/{circle}/report/{circle}_{month}_upaid_new.xlsx")

In [116]:
# pivot_table.plot(kind='pie',
#             subplots=True,
#               title='Population (1955-2020)',autopct='%1.0f%%')

Catvc=df[(df.bilable_flag == True) & (df.gov_flag == False)].pivot_table(index=['Circle_Name','Division_Name',], columns=['Cat_vc'],values=['ACCT_ID','CURRENT_ASSESSMENT','TOTAL_OUTSTANDING','TOTAL_PAY_AMT','paid_status','LOAD'],
                    aggfunc={'ACCT_ID':'count','CURRENT_ASSESSMENT':np.sum,'TOTAL_OUTSTANDING':np.sum,'TOTAL_PAY_AMT':np.sum,'paid_status':np.sum,'LOAD':np.sum},fill_value=0,margins=True,margins_name='Total' )


In [131]:
# pivot_table.plot(kind='pie',
#             subplots=True,
#               title='Population (1955-2020)',autopct='%1.0f%%')

div_cat=df[(df.bilable_flag == True) & (df.gov_flag == False)].pivot_table(index=['Circle_Name','Division_Name',], columns=['Cat'],values=['ACCT_ID','CURRENT_ASSESSMENT','TOTAL_OUTSTANDING','TOTAL_PAY_AMT','paid_status','LOAD'],
                    aggfunc={'ACCT_ID':'count','CURRENT_ASSESSMENT':np.sum,'TOTAL_OUTSTANDING':np.sum,'TOTAL_PAY_AMT':np.sum,'paid_status':np.sum,'LOAD':np.sum},fill_value=0,margins=True,margins_name='Total' )


In [52]:
# pivot_table.plot(kind='pie',
#             subplots=True,
#               title='Population (1955-2020)',autopct='%1.0f%%')

ots_1=df[(df.bilable_flag == True) & (df.LPSC >1) ].pivot_table(index=['Circle_Name','Division_Name'], columns=['ots_vc'],values=['ACCT_ID','arrear_withot_Lpsc'],
                    aggfunc={'ACCT_ID':'count','arrear_withot_Lpsc':np.sum},fill_value=0,margins=True,margins_name='Total' )


In [53]:
# pivot_table.plot(kind='pie',
#             subplots=True,
#               title='Population (1955-2020)',autopct='%1.0f%%')

ots_500=df[(df.bilable_flag == True) & (df.LPSC >= 500) ].pivot_table(index=['Circle_Name','Division_Name','ARREAR_FILTER'], columns=['ots_vc'],values=['ACCT_ID','arrear_withot_Lpsc'],
                    aggfunc={'ACCT_ID':'count','arrear_withot_Lpsc':np.sum},fill_value=0,margins=True,margins_name='Total' )


In [146]:
# df['discount_Nov'] = None
#     df['discount_Dec(1-15)'] = None
#     df['Registration_amount'] = None
#     df['Registration_amount'] = 0.3*df['TOTAL_OUTSTANDING']
#     df['Amount_to_be_paid_after_disc_Nov'] = None
#     df['Amount_to_be_paid_after_disc_dec(1-15)']
feeder = ["RANI BAZAR_35521114103RIR","TOWN III_35521110170TO3",'BALESHWAR GANJ_35523410632BWJ',"TARAB GANJ TOWN_35523420638TTW ","TAHSEEL_35523310390THL","DHANEPUR TOWN_35523110163DTW","DHARAMPUR_35513110252DRP","LIFT CANAL_35513110176LCL","EAST_35513110180EAT","SADULLA NAGAR_35513130186SLR","IPDS 1_35513131017IP1","SEMRI DEHAT_35492210434SDT","SIRSIYA_35492210434SRA","PACHDEVARI_35492220437PHI","JAMUNHA_35492210436JUA","TIWARI GAON_35492210436TRN","RURAL_35492110453RUL","RAHVA_35491110455RAA","NEJABHAAR_35492220555NAR","MAHARAJGANJ_35492220556MRJ","JAITAPUR_35492150460JAT"]

In [50]:
div_highloss_nov = df[(df.bilable_flag == True) & (df.FEEDER.isin(feeder))].pivot_table(
    index=['Circle_Name', 'Division_Name','FEEDER'], values=['ACCT_ID','CONSUMPTION_CURR_MNTH','CURRENT_ASSESSMENT','TOTAL_PAY_AMT','LOAD'],
    aggfunc={'ACCT_ID':'count','LOAD':np.sum,'CONSUMPTION_CURR_MNTH':np.sum,'CURRENT_ASSESSMENT':np.sum,'TOTAL_PAY_AMT':np.sum}, fill_value=0
)

In [51]:
div_highloss_nov.to_excel(f"{month}/{circle}/report/{circle}_{month}_highloss_nov.xlsx")

In [55]:
# pivot_table.plot(kind='pie',
#             subplots=True,
#               title='Population (1955-2020)',autopct='%1.0f%%')

ots_detail=df[(df.bilable_flag == True) & (df.LPSC >= 100) ].pivot_table(index=['Circle_Name','Division_Name'], columns=['ots_vc'],values=['ACCT_ID','TOTAL_OUTSTANDING','discount_Nov','Registration_amount','Amount_to_be_paid_after_disc_Nov'],
                    aggfunc={'ACCT_ID':'count','discount_Nov':np.sum,'TOTAL_OUTSTANDING':np.sum,'Registration_amount':np.sum,'Amount_to_be_paid_after_disc_Nov':np.sum},fill_value=0,margins=True,margins_name='Total' )


In [31]:
# # pivot_table.plot(kind='pie',
# #             subplots=True,
# #               title='Population (1955-2020)',autopct='%1.0f%%')

# Csc_count=df[(df.PAYMENT_SRCE == "EGSIL001") & (df.paid_status==1)].pivot_table(index=['Circle_Name','Division_Name'],values=['ACCT_ID','TOTAL_PAY_AMT','paid_status'],
#                     aggfunc={'ACCT_ID':'count','TOTAL_PAY_AMT':np.sum,'paid_status':np.sum},fill_value=0,margins=True,margins_name='Total' )


In [55]:
# # pivot_table.plot(kind='pie',
# #             subplots=True,
# #               title='Population (1955-2020)',autopct='%1.0f%%')

# Catvc_acc=df[(df.bilable_flag == True)].pivot_table(index=['Circle_Name','Division_Name'], columns=['Cat'],values=['ACCT_ID'],
#                     aggfunc={'ACCT_ID':'count'},fill_value=0,margins=True,margins_name='Total' )


In [47]:
# pivot_table.plot(kind='pie',
#             subplots=True,
#               title='Population (1955-2020)',autopct='%1.0f%%')

div_cat_connection=df[(df.bilable_flag == True)&(df.newconection== 0)].pivot_table(index=['Circle_Name','Division_Name'], columns=['gov_flag','Cat'],values=['ACCT_ID','LOAD','CONSUMPTION_CURR_MNTH','CURRENT_ASSESSMENT','TOTAL_OUTSTANDING','TOTAL_PAY_AMT','paid_status'],
                    aggfunc={'ACCT_ID':'count','LOAD':np.sum,'CONSUMPTION_CURR_MNTH':np.sum,'CURRENT_ASSESSMENT':np.sum,'TOTAL_OUTSTANDING':np.sum,'TOTAL_PAY_AMT':np.sum,'paid_status':np.sum},fill_value=0 )


In [57]:
# pivot_table.plot(kind='pie',
#             subplots=True,
#               title='Population (1955-2020)',autopct='%1.0f%%')

arrear_paid=df[(df.bilable_flag == True)].pivot_table(index=['Circle_Name','Division_Name'], columns=['gov_flag'],values=['TOTAL_PAY_AMT','paid_arrear'],
                    aggfunc={'paid_arrear':np.sum,'TOTAL_PAY_AMT':np.sum},fill_value=0 )


In [117]:
# pivot_table.plot(kind='pie',
#             subplots=True,
#               title='Population (1955-2020)',autopct='%1.0f%%')

div=df[(df.bilable_flag == True)].pivot_table(index=['Circle_Name','Division_Name'], columns=[],values=['ACCT_ID','LOAD','CONSUMPTION_CURR_MNTH','CURRENT_ASSESSMENT','TOTAL_OUTSTANDING','TOTAL_PAY_AMT','paid_status'],
                    aggfunc={'ACCT_ID':'count','LOAD':np.sum,'CONSUMPTION_CURR_MNTH':np.sum,'CURRENT_ASSESSMENT':np.sum,'TOTAL_OUTSTANDING':np.sum,'TOTAL_PAY_AMT':np.sum,'paid_status':np.sum},fill_value=0 )


In [134]:
# pivot_table.plot(kind='pie',
#             subplots=True,
#               title='Population (1955-2020)',autopct='%1.0f%%')

div_csc=df[(df.bilable_flag == True) & (df.paid_status==1) & (df.PAYMENT_SRCE=='EGSIL001')].pivot_table(index=['Circle_Name','Division_Name'], columns=['LAST_PAY_DATE'],values=['TOTAL_PAY_AMT'],
                    aggfunc={'TOTAL_PAY_AMT':np.sum},fill_value=0 )


In [136]:
# pivot_table.plot(kind='pie',
#             subplots=True,
#               title='Population (1955-2020)',autopct='%1.0f%%')

div_csc_other=df[(df.bilable_flag == True) & (df.paid_status==1) & (df.PAYMENT_SRCE=='EGSIL001')].pivot_table(index=['LAST_PAY_DATE'], columns=['Circle_Name','Division_Name'],values=['TOTAL_PAY_AMT'],
                    aggfunc={'TOTAL_PAY_AMT':np.sum},fill_value=0 )


In [118]:
zone_arrear_cat_vc =df[(df.gov_flag == False) &(df.bilable_flag == True) ].pivot_table(index=['Circle_Name','Division_Name'], columns=['ARREAR_FILTER'], values=['ACCT_ID','CURRENT_ASSESSMENT','TOTAL_OUTSTANDING','TOTAL_PAY_AMT','paid_status','Neverpaid_status'],
                    aggfunc={'ACCT_ID':'count','TOTAL_OUTSTANDING':np.sum,'CURRENT_ASSESSMENT':np.sum,
                            'TOTAL_PAY_AMT':np.sum,'paid_status':np.sum,'Neverpaid_status':np.sum},fill_value=0,margins=True,margins_name='Total' )

ValueError: No objects to concatenate

In [119]:
zone_cat_cat =df[(df.gov_flag == False) &(df.bilable_flag == True) ].pivot_table(index=['Circle_Name','Division_Name'], columns=['Cat_vc'], values=['ACCT_ID','CURRENT_ASSESSMENT','TOTAL_OUTSTANDING','TOTAL_PAY_AMT','paid_status','Neverpaid_status'],
                    aggfunc={'ACCT_ID':'count','TOTAL_OUTSTANDING':np.sum,'CURRENT_ASSESSMENT':np.sum,
                            'TOTAL_PAY_AMT':np.sum,'paid_status':np.sum,'Neverpaid_status':np.sum},fill_value=0,margins=True,margins_name='Total' )

In [147]:
feeder = ["RANI BAZAR_35521114103RIR","TOWN III_35521110170TO3","TARAB GANJ TOWN_35523420638TTW","TAHSEEL_35523310390THL","DHANEPUR TOWN_35523110163DTW","DHARAMPUR_35513110252DRP","LIFT CANAL_35513110176LCL","EAST_35513110180EAT","SADULLA NAGAR_35513130186SLR","IPDS 1_35513131017IP1","SEMRI DEHAT_35492210434SDT","SIRSIYA_35492210434SRA","PACHDEVARI_35492220437PHI","JAMUNHA_35492210436JUA","TIWARI GAON_35492210436TRN","RURAL_35492110453RUL","RAHVA_35491110455RAA","NEJABHAAR_35492220555NAR","MAHARAJGANJ_35492220556MRJ","JAITAPUR_35492150460JAT"]
feeder_report =df[(df.bilable_flag == True) & (df['FEEDER'].isin(feeder))].pivot_table(index=['Circle_Name','Division_Name','FEEDER'], values=['ACCT_ID','LOAD','CONSUMPTION_CURR_MNTH','CURRENT_ASSESSMENT','TOTAL_PAY_AMT','paid_status','TOTAL_OUTSTANDING'],
                    aggfunc={'ACCT_ID':'count','TOTAL_OUTSTANDING':np.sum,'CURRENT_ASSESSMENT':np.sum,'LOAD':np.sum,'CONSUMPTION_CURR_MNTH':np.sum,
                            'TOTAL_PAY_AMT':np.sum,'paid_status':np.sum},fill_value=0,margins=True)

In [148]:
# feeder = ["RANI BAZAR_35521114103RIR","TOWN III_35521110170TO3","TARAB GANJ TOWN_35523420638TTW ","TAHSEEL_35523310390THL","DHANEPUR TOWN_35523110163DTW","DHARAMPUR_35513110252DRP","LIFT CANAL_35513110176LCL","EAST_35513110180EAT","SADULLA NAGAR_35513130186SLR","IPDS 1_35513131017IP1","SEMRI DEHAT_35492210434SDT","SIRSIYA_35492210434SRA","PACHDEVARI_35492220437PHI","JAMUNHA_35492210436JUA","TIWARI GAON_35492210436TRN","RURAL_35492110453RUL","RAHVA_35491110455RAA","NEJABHAAR_35492220555NAR","MAHARAJGANJ_35492220556MRJ","JAITAPUR_35492150460JAT"]
feeder_report_load =df[(df.bilable_flag == True) & (df['FEEDER'].isin(feeder))].pivot_table(index=['Circle_Name','Division_Name','SUBSTATION','FEEDER'],columns = ['Cat','conection_month'], values=['ACCT_ID','LOAD'],
                    aggfunc={'ACCT_ID':'count','LOAD':np.sum},fill_value=0,margins=True)

In [149]:
# feeder = ["RANI BAZAR_35521114103RIR","TOWN III_35521110170TO3","TARAB GANJ TOWN_35523420638TTW ","TAHSEEL_35523310390THL","DHANEPUR TOWN_35523110163DTW","DHARAMPUR_35513110252DRP","LIFT CANAL_35513110176LCL","EAST_35513110180EAT","SADULLA NAGAR_35513130186SLR","IPDS 1_35513131017IP1","SEMRI DEHAT_35492210434SDT","SIRSIYA_35492210434SRA","PACHDEVARI_35492220437PHI","JAMUNHA_35492210436JUA","TIWARI GAON_35492210436TRN","RURAL_35492110453RUL","RAHVA_35491110455RAA","NEJABHAAR_35492220555NAR","MAHARAJGANJ_35492220556MRJ","JAITAPUR_35492150460JAT"]
feeder_report_idfrdf =df[(df.bilable_flag == True) & (df['FEEDER'].isin(feeder)) & ((df.METER_READ_REMARK== 'RDF') | (df.METER_READ_REMARK== 'IDF'))].pivot_table(index=['Circle_Name','Division_Name','SUBSTATION','FEEDER'],columns = ['METER_READ_REMARK'], values=['ACCT_ID'],
                    aggfunc={'ACCT_ID':'count'},fill_value=0,margins=True)

In [152]:
feeder_report_idfrdf.to_excel(f"nov2023/{circle}/report/{circle}_feeder_idfrdf_nov.xlsx")

In [153]:
feeder_report_load.to_excel(f"nov2023/{circle}/report/{circle}_feeder_load_nov.xlsx")

In [120]:
graphvc.to_excel(f"{month}/{circle}/report/{circle}_graph_dec_new.xlsx")

In [127]:
Catvc.to_excel(f"{month}/{circle}/report/{circle}_Cat_VC_dec_new.xlsx")

In [133]:
div_cat.to_excel(f"{month}/{circle}/report/{circle}_div_cat_dec_new.xlsx")

In [128]:
div.to_excel(f"{month}/{circle}/report/{circle}_Cat_VC_dec_new.xlsx")

In [117]:
meter_read_remark.to_excel(f"{month}/{circle}/report/{circle}_read_remark_nov.xlsx")

In [118]:
BILL_BASIS.to_excel(f"{month}/{circle}/report/{circle}_BILL_BASIS_nov.xlsx")

In [134]:
Sbm.to_excel(f"{month}/{circle}/report/{circle}_sbm_nov.xlsx")

In [43]:
rdf_sdo.to_excel(f"{month}/{circle}/report/{circle}_nov_rdf_sdo.xlsx")

In [125]:
div_cat_connection.to_excel(f"{month}/{circle}/report/{circle}_nov_cat_conn.xlsx")

NameError: name 'div_cat_connection' is not defined

In [121]:
zone_arrear_cat_vc.to_excel(f"{month}/{circle}/report/{circle}_arrear_cat_dec.xlsx")

NameError: name 'zone_arrear_cat_vc' is not defined

In [122]:
zone_cat_cat.to_excel(f"{month}/{circle}/report/{circle}_arrear_catVC_dec.xlsx")

In [75]:
ots_1.to_excel(f"{month}/{circle}/report/{circle}_cat_nov_ots_1.xlsx")

In [76]:
ots_500.to_excel(f"{month}/{circle}/report/{circle}_cat_nov_ots_500.xlsx")

In [77]:
div_csc_other.to_excel(f"{month}/{circle}/report/{circle}_CSC_nov_other.xlsx")

NameError: name 'div_csc_other' is not defined

In [123]:
arrear_paid.to_excel(f"{month}/{circle}/report/{circle}_arrear_paid_dec.xlsx")

NameError: name 'arrear_paid' is not defined

In [79]:
feeder_report.to_excel(f"{month}/{circle}/report/{circle}_feeder_nov.xlsx")

In [124]:
zone_graph.to_excel(f"{month}/{circle}/report/{circle}_graph_dec.xlsx")

NameError: name 'zone_graph' is not defined

In [44]:
df1.shape

(9878, 55)

In [43]:
df1.to_excel(f"{month}/{circle}_oct_10kw.xlsx")

In [81]:
div_cat.to_excel(f"{month}/{circle}/report/{circle}_divCAT_nov.xlsx")

In [ ]:
import ftplib

path = 'ftp://mvftpreport@ftp.uppclonline.com/CSV_MASTER/25092023/MASTER_MVVNL_DIV354921_25092023.csv.gz'
filename = 'MASTER_MVVNL_DIV354921_25092023.csv.gz'  
# ftp://mvftpreport@ftp.uppclonline.com/CSV_MASTER/25092023/MASTER_MVVNL_DIV354921_25092023.csv.gz

ftp = ftplib.FTP("Server IP") 
ftp.login("UserName", "Password") 
ftp.cwd(path)
ftp.retrbinary("RETR " + filename, open(filename, 'wb').write)
ftp.quit()

In [1]:
import wget
link = 'ftp://mvftpreport@ftp.uppclonline.com/CSV_MASTER/25092023/MASTER_MVVNL_DIV354921_25092023.csv.gz'
wget.download(link)

ModuleNotFoundError: No module named 'wget'